# Redes Neuronales Artificales

# Cómo importar las librerías


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

# Importar el data set


In [2]:
dataset = pd.read_csv('./Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
X_2 = dataset.iloc[:, 3:13].values

# Parte 1 - Pre procesado de datos

# Codificar datos categóricos

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) # Permite categorizar variables en formato de texto
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
) 

# Permite crear columnas para cada una de las variables dummy que hemos categorizado
# esnecesario eliminar columnas para evitar multicolinealidad
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

# Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Escalado de variables

In [5]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

# Importar Keras y librerías adicionales


In [6]:
import keras
from keras.models import Sequential #inicializar los parámetros de la red neuronal
from keras.layers import Dense #crear las capas de la red

Using TensorFlow backend.
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\fra

# Inicializar la RNA

In [30]:
classifier = Sequential() #inicializar los parámetros de la red neuronal

# Añadir las capas de entrada y primera capa oculta


In [31]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))

# Añadir la segunda capa oculta

In [32]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

In [33]:
classifier.add(Dropout(rate = 0.1))

# Añadir la capa de salida

In [34]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA

In [35]:
classifier.compile(optimizer = "adam", loss = "mae", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento

In [36]:
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 0s 49us/step - loss: 0.2777 - accuracy: 0.7956
Epoch 2/100
8000/8000 [==============================] - 0s 38us/step - loss: 0.2017 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 0s 37us/step - loss: 0.1935 - accuracy: 0.8025
Epoch 4/100
8000/8000 [==============================] - 0s 36us/step - loss: 0.1785 - accuracy: 0.8324
Epoch 5/100
8000/8000 [==============================] - 0s 36us/step - loss: 0.1724 - accuracy: 0.8356
Epoch 6/100
8000/8000 [==============================] - 0s 36us/step - loss: 0.1703 - accuracy: 0.8353
Epoch 7/100
8000/8000 [==============================] - 0s 37us/step - loss: 0.1683 - accuracy: 0.8360
Epoch 8/100
8000/8000 [==============================] - 0s 36us/step - loss: 0.1668 - accuracy: 0.8372
Epoch 9/100
8000/8000 [==============================] - 0s 36us/step - loss: 0.1662 - accuracy: 0.8372
Epoch 10/100
8000/8000 [==============================] - 0s 36u

# Parte 3 - Evaluar el modelo y calcular predicciones finales

# Predicción de los resultados con el Conjunto de Testing

In [13]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

# Tarea

In [14]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [15]:
data = pd.DataFrame(index=[0],columns=dataset.columns)

In [16]:
data.iloc[0,3:-1] = np.array([600,'France','Male',40,3,60000,2,1,1,50000])

In [17]:
data = data.dropna(axis=1)

In [18]:
X_h = data.values

In [19]:
X_h[:, 1] = labelencoder_X_1.fit_transform(X_h[:, 1])
X_h[:, 2] = labelencoder_X_2.fit_transform(X_h[:, 2])
X_h = onehotencoder.transform(X_h)
X_h = X_h[:, 1:]

In [20]:
y_pred_tarea  = classifier.predict(sc_X.transform(X_h))
y_pred_tarea = (y_pred_tarea>0.5)

# Elaborar una matriz de confusión

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [22]:
cm

array([[1518,   77],
       [ 199,  206]], dtype=int64)

## Parte 4 - Evaluar, mejorar y Ajustar la RNA


### Evaluar la **RNA**


In [40]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [24]:
def build_classifier():
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [25]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.5s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.5s finished


In [26]:
mean = accuracies.mean()
variance = accuracies.std()

In [27]:
accuracies

array([0.78625   , 0.79000002, 0.80000001, 0.78250003, 0.81625003,
       0.81      , 0.78750002, 0.79374999, 0.79874998, 0.79500002])

### Mejorar la RNA

#### Regularización de Dropout para evitar el *overfitting*


### Ajustar la *RNA*


In [50]:
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from sklearn.decomposition import PCA

In [77]:
def build_classifier(optimizer,units):
    classifier = Sequential()
    classifier.add(Dense(units = units, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
    classifier.add(Dense(units = units, kernel_initializer = "uniform",  activation = "relu"))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
    classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
    return classifier


In [78]:
classifier = KerasClassifier(build_fn = build_classifier)

In [83]:
parameters = {
    'units':[6,12,24],
    'batch_size' : [25,64],
    'nb_epoch' : [100, 500], 
    'optimizer' : ['SGD', 'Adamax']
}

In [84]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv=10,n_jobs=-1, verbose = 1)
grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(best_parameters)
print(best_accuracy)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


C:\Anaconda3\envs\tensorflow\lib\site-packages\joblib\externals\loky\process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/1
8000/8000 [==============================] - 1s 159us/step - loss: 0.6048 - accuracy: 0.7956
{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'SGD', 'units': 6}
0.796
